# Normalize Lithology CSVs

In [1]:
import sys
sys.path.append('../scripts/')
import glob
import re

import pandas as pd
import numpy as np

from normalize_data import (
    normalize_columns,
    csv_cleanup,
    update_metadata,
    fetch_unique_column_names,
    append_set,
    filter_existing_set,
    add_missing_columns
)

In [2]:
metadata_file = 'cleaned_data/metadata/lims_lithology_changes.csv'
clean_data_path = 'cleaned_data/Lithology_CSV'

## Normalize Lithology columns

Normalize all the Lithology Prefix, Principal Name, Suffix, Minor Prefix, Minor Name, Minor Suffix columns to have the same names.

In [4]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,empty_rows_columns,remove_identical_rows,add_expedition_section_cols,update_sample_col,update_top_bottom,add_missing_cols
0,361_macroscopic_U1474D.csv,Lithology_CSV/361_macroscopic_U1474D.csv,False,False,False,True,False,True
1,323 Core Description Template_U1341A.csv,Lithology_CSV/323 Core Description Template_U1...,False,False,False,False,False,True
2,361_macroscopic_U1479C.csv,Lithology_CSV/361_macroscopic_U1479C.csv,False,False,False,True,False,True
3,340_sediment_U1393A.csv,Lithology_CSV/340_sediment_U1393A.csv,False,False,False,True,False,True
4,339_sediment_U1386A.csv,Lithology_CSV/339_sediment_U1386A.csv,False,False,False,False,False,True


### Get Lithology columns

In [5]:
columns_all = set()
    
res=[fetch_unique_column_names(f"{clean_data_path}/{file}", columns_all) for file in metadata['file']] 

In [6]:
def remove_symbols_from_set(old_set):
    return filter_existing_set(old_set, r"(.*?%)|(.*?\+)")

def remove_minor_from_set(old_set):
    return filter_existing_set(old_set, r"(.*?min .*?)|(.*?minor .*?)|(Lithology 2.*?)")

In [7]:
lith_all = set()
prefix_all = set()
suffix_all = set()
principal_all = set()
modifier_all = set()
minor_all = set()
color_all = set()

append_set(lith_all, r".*?lith.*?", columns_all)

append_set(modifier_all, r".*?modifier.*?", columns_all)

append_set(suffix_all, r".*?suffix.*?", columns_all)
append_set(suffix_all, r".*?modifier after.*?", columns_all)

append_set(prefix_all, r".*?prefix.*?", columns_all)
append_set(prefix_all, r".*?modifier before.*?", columns_all)

append_set(principal_all, r".*?principal.*?", columns_all)
append_set(principal_all, r"^Lithology$", columns_all)

append_set(minor_all, r"(.*?min lith.*?)|(.*?minor modifier.*?)|(.*?minor suffix.*?)|(Lithology 2.*?)", columns_all)

append_set(color_all, r".*?color.*?", columns_all)


In [8]:
minor_prefix = set()
minor_principal = set()
minor_suffix = set()

suffix = remove_symbols_from_set(suffix_all)
suffix = remove_minor_from_set(suffix)

prefix = remove_symbols_from_set(prefix_all)
prefix = remove_minor_from_set(prefix)

principal = remove_symbols_from_set(principal_all)
principal = remove_minor_from_set(principal)
principal = filter_existing_set(principal, r".*?modifier.*?")

minor = remove_symbols_from_set(minor_all)

append_set(minor_prefix, r".*?prefix.*?", minor)
append_set(minor_principal, r".*?principal.*?", minor)
append_set(minor_suffix, r"(.*?suffix.*?)|(.*?modifier after.*?)", minor)
minor_principal = filter_existing_set(minor_principal, r".*?modifier.*?")


In [10]:
lith_all

{'2ND lithic roundness',
 '3RD lithic roundness',
 'Amount of ash in lithology',
 'Amount of ash in lithology rank [read only]',
 'Bedding thickness of MAJOR lithology',
 'Clast 3 -other lithology [%]',
 'Clast 3 -other lithology grain size',
 'Clast 7 -other lithology [%]',
 'Clast 7 -other lithology grain size',
 'Clast lith. abund. sum [%]',
 'Clast lithology',
 'Clast lithology comment',
 'Complete lithology name',
 'DOMINANT lithic roundness',
 'GRAVEL SIZE CLASTLITHOLOGY',
 'Interbed lithology',
 'LITH 1 Ash abundance name',
 'LITH 1 Ash abundance rank (read only)',
 'LITH 1 Bioturbation intensity',
 'LITH 1 Bioturbation intensity rank [read only]',
 'LITH 1 Clast 1 igneous -  grain size',
 'LITH 1 Clast 1 igneous -  lithology',
 'LITH 1 Clast 1 igneous -  roundness',
 'LITH 1 Clast 1 igneous [%]',
 'LITH 1 Clast 2 sediment - grain size',
 'LITH 1 Clast 2 sediment - roundness',
 'LITH 1 Clast 2 sediment [%]',
 'LITH 1 Clast 3 -other LITH [%]',
 'LITH 1 Clast 3 metamorphic - grain

In [11]:
modifier_all

{'Major  prefix modifier before principal',
 'Major ModifierPrefixModifier Before Principal',
 'Major modifier before principal',
 'Major prefix modifier before principal',
 'Matrix modifier',
 'Minor ModifierSuffixModifier After Principal',
 'Minor modifier after principal',
 'Minor suffix modifier after principal'}

In [12]:
prefix_all

{'LITH 1 prefix+name [read only]',
 'LITH 1 prefix+name+suffix [read only]',
 'LITH 2 prefix+name [read only]',
 'LITH 2 prefix+name+suffix [read only]',
 'Lith. prefix+name+suffix',
 'Lithology 1 prefix',
 'Lithology 2 prefix',
 'Lithology PREFIX',
 'Lithology Prefix',
 'Lithology prefix',
 'Lithology prefix + name',
 'Lithology prefix+name [read only]',
 'Lithology prefix+name+suffix [read only]',
 'MAJ Lith. Prefix',
 'MAJ Lith. Prefix abundance Plot [%]',
 'MAJ Lith. prefix',
 'MAJ Lith. prefix + name',
 'MAJ Lith. prefix abundance plot [%]',
 'MAJ Lith. prefix+name [read only]',
 'MAJ Lith. prefix+name+suffix',
 'MAJ Lith. prefix+name+suffix [read only]',
 'MAJ Lithology prefix A',
 'MIN Lith. Prefix',
 'MIN Lith. prefix',
 'MIN Lith. prefix + name',
 'MIN Lith. prefix+name+suffix',
 'Major  prefix modifier before principal',
 'Major ModifierPrefixModifier Before Principal',
 'Major modifier before principal',
 'Major prefix modifier before principal',
 'PREFIX',
 'Prefix'}

In [13]:
prefix

{'Lithology 1 prefix',
 'Lithology PREFIX',
 'Lithology Prefix',
 'Lithology prefix',
 'MAJ Lith. Prefix',
 'MAJ Lith. prefix',
 'MAJ Lithology prefix A',
 'Major  prefix modifier before principal',
 'Major ModifierPrefixModifier Before Principal',
 'Major modifier before principal',
 'Major prefix modifier before principal',
 'PREFIX',
 'Prefix'}

In [14]:
suffix_all

{'LITH 1 prefix+name+suffix [read only]',
 'LITH 2 prefix+name+suffix [read only]',
 'Lith. prefix+name+suffix',
 'Lithology 1 suffix',
 'Lithology 2 suffix',
 'Lithology SUFFIX',
 'Lithology Suffix',
 'Lithology name+suffix',
 'Lithology name+suffix [read only]',
 'Lithology prefix+name+suffix [read only]',
 'Lithology suffix',
 'MAJ Lith. Suffix',
 'MAJ Lith. name+suffix',
 'MAJ Lith. prefix+name+suffix',
 'MAJ Lith. prefix+name+suffix [read only]',
 'MAJ Lith. suffix',
 'MAJ Lithology suffix C',
 'MIN Lith. Suffix',
 'MIN Lith. name+suffix',
 'MIN Lith. prefix+name+suffix',
 'MIN Lith. suffix',
 'Minor ModifierSuffixModifier After Principal',
 'Minor modifier after principal',
 'Minor suffix modifier after principal',
 'SUFFIX',
 'Suffix'}

In [15]:
suffix 

{'Lithology 1 suffix',
 'Lithology SUFFIX',
 'Lithology Suffix',
 'Lithology suffix',
 'MAJ Lith. Suffix',
 'MAJ Lith. suffix',
 'MAJ Lithology suffix C',
 'SUFFIX',
 'Suffix'}

In [16]:
principal_all

{'LITHOLOGY',
 'Lithology',
 'Lithology 1 principal name',
 'Lithology 2 principal name',
 'Lithology Principal name',
 'Lithology principal name',
 'MAJ Lith. Principal name',
 'MAJ Lith. Principal name abundance Plot [%]',
 'MAJ Lith. principal name',
 'MAJ Lith. principal name abundance plot [%]',
 'MAJ Principal lithology B',
 'MIN Lith. Principal name',
 'MIN Lith. principal name',
 'Major  prefix modifier before principal',
 'Major ModifierPrefixModifier Before Principal',
 'Major modifier before principal',
 'Major prefix modifier before principal',
 'Minor ModifierSuffixModifier After Principal',
 'Minor modifier after principal',
 'Minor suffix modifier after principal',
 'PRINCIPAL Lithology',
 'PRINCIPALLITHOLOGY',
 'Principal  lithology',
 'Principal Lithology',
 'Principal lithology',
 'Principal lithology name',
 'PrincipalLithology Name'}

In [17]:
principal

{'LITHOLOGY',
 'Lithology',
 'Lithology 1 principal name',
 'Lithology Principal name',
 'Lithology principal name',
 'MAJ Lith. Principal name',
 'MAJ Lith. principal name',
 'MAJ Principal lithology B',
 'PRINCIPAL Lithology',
 'PRINCIPALLITHOLOGY',
 'Principal  lithology',
 'Principal Lithology',
 'Principal lithology',
 'Principal lithology name',
 'PrincipalLithology Name'}

In [18]:
minor_all

{'Lithology 2 prefix',
 'Lithology 2 principal name',
 'Lithology 2 suffix',
 'MIN Lith. Prefix',
 'MIN Lith. Principal name',
 'MIN Lith. Suffix',
 'MIN Lith. abundance [%]',
 'MIN Lith. ave. grain size ',
 'MIN Lith. ave. grain size rank',
 'MIN Lith. color',
 'MIN Lith. max. grain size ',
 'MIN Lith. max. grain size rank',
 'MIN Lith. name+suffix',
 'MIN Lith. prefix',
 'MIN Lith. prefix + name',
 'MIN Lith. prefix+name+suffix',
 'MIN Lith. principal name',
 'MIN Lith. suffix',
 'MIN lith color (simple)',
 'MIN lithology comment',
 'Minor ModifierSuffixModifier After Principal',
 'Minor modifier after principal',
 'Minor suffix modifier after principal'}

In [19]:
minor

{'Lithology 2 prefix',
 'Lithology 2 principal name',
 'Lithology 2 suffix',
 'MIN Lith. Prefix',
 'MIN Lith. Principal name',
 'MIN Lith. Suffix',
 'MIN Lith. ave. grain size ',
 'MIN Lith. ave. grain size rank',
 'MIN Lith. color',
 'MIN Lith. max. grain size ',
 'MIN Lith. max. grain size rank',
 'MIN Lith. prefix',
 'MIN Lith. principal name',
 'MIN Lith. suffix',
 'MIN lith color (simple)',
 'MIN lithology comment',
 'Minor ModifierSuffixModifier After Principal',
 'Minor modifier after principal',
 'Minor suffix modifier after principal'}

In [20]:
minor_prefix

{'Lithology 2 prefix', 'MIN Lith. Prefix', 'MIN Lith. prefix'}

In [21]:
minor_principal

{'Lithology 2 principal name',
 'MIN Lith. Principal name',
 'MIN Lith. principal name'}

In [22]:
minor_suffix

{'Lithology 2 suffix',
 'MIN Lith. Suffix',
 'MIN Lith. suffix',
 'Minor ModifierSuffixModifier After Principal',
 'Minor modifier after principal',
 'Minor suffix modifier after principal'}

In [41]:
color_all

{'COLOR',
 'Clast color',
 'Color',
 'Color (name)',
 'Color code',
 'Color(name)',
 'LITH 1 color',
 'LITH 2 color',
 'Lithology color',
 'Lithology color (Munsell)',
 'Lithology color (simple)',
 'MAJ Lith. color',
 'MAJ Lithology color',
 'MAJ lith color (simple)',
 'MIN Lith. color',
 'MIN lith color (simple)'}

### Normalize Lithology columns

In [29]:
metadata = pd.read_csv(metadata_file)

In [30]:
def normalize_top_bottom(file):
    path = f"{clean_data_path}/{file}"
    content = pd.read_csv(path, dtype=str)
    columns = list(content.columns)
    
    normalized_cols = normalize_columns(prefix, 'Lithology Prefix', columns)
    normalized_cols = normalize_columns(principal, 'Lithology Principal Name', normalized_cols)
    normalized_cols = normalize_columns(suffix, 'Lithology Suffix', normalized_cols)

    normalized_cols = normalize_columns(minor_prefix, 'Minor Lithology Prefix', normalized_cols)
    normalized_cols = normalize_columns(minor_principal, 'Minor Lithology Name', normalized_cols)
    normalized_cols = normalize_columns(minor_suffix, 'Minor Lithology Suffix', normalized_cols)

    changed = columns != normalized_cols
    
    if changed:
        content.columns = normalized_cols
        content = csv_cleanup(content, path)
        content.to_csv(path, index=False)

    return changed

change_columns = [normalize_top_bottom(file) for file in metadata['file']] 

### Update metadata

In [31]:
dict = {"update_lithology": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,path,empty_rows_columns,remove_identical_rows,add_expedition_section_cols,update_sample_col,update_top_bottom,add_missing_cols,update_lithology
0,361_macroscopic_U1474D.csv,Lithology_CSV/361_macroscopic_U1474D.csv,False,False,False,True,False,True,True
1,323 Core Description Template_U1341A.csv,Lithology_CSV/323 Core Description Template_U1...,False,False,False,False,False,True,True
2,361_macroscopic_U1479C.csv,Lithology_CSV/361_macroscopic_U1479C.csv,False,False,False,True,False,True,True
3,340_sediment_U1393A.csv,Lithology_CSV/340_sediment_U1393A.csv,False,False,False,True,False,True,True
4,339_sediment_U1386A.csv,Lithology_CSV/339_sediment_U1386A.csv,False,False,False,False,False,True,True


In [32]:
new_metadata.to_csv(metadata_file, index=False)

## Add missing Lithology columns

In [33]:
metadata = pd.read_csv(metadata_file)

In [34]:
normalized_columns = [
    'Lithology Prefix',
    'Lithology Principal Name',
    'Lithology Suffix',
    'Minor Lithology Prefix',
    'Minor Lithology Name',
    'Minor Lithology Suffix',
]

In [35]:
change_columns = [add_missing_columns(f"{clean_data_path}/{file}", normalized_columns) for file in metadata['file']] 

In [36]:
dict = {"add_missing_lith_cols": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,path,empty_rows_columns,remove_identical_rows,add_expedition_section_cols,update_sample_col,update_top_bottom,add_missing_cols,update_lithology,add_missing_lith_cols
0,361_macroscopic_U1474D.csv,Lithology_CSV/361_macroscopic_U1474D.csv,False,False,False,True,False,True,True,True
1,323 Core Description Template_U1341A.csv,Lithology_CSV/323 Core Description Template_U1...,False,False,False,False,False,True,True,True
2,361_macroscopic_U1479C.csv,Lithology_CSV/361_macroscopic_U1479C.csv,False,False,False,True,False,True,True,True
3,340_sediment_U1393A.csv,Lithology_CSV/340_sediment_U1393A.csv,False,False,False,True,False,True,True,True
4,339_sediment_U1386A.csv,Lithology_CSV/339_sediment_U1386A.csv,False,False,False,False,False,True,True,False


In [37]:
new_metadata.to_csv(metadata_file, index=False)